In [11]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
import tensorboard

# import some common libraries
import numpy as np
import os, json, cv2, random

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

In [1]:
image = cv2.imread('/home/tarobben/scratch/RCNN/PFO_png/1006_70/1006_70_slice_000.png')
cv2.imshow('lol', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

NameError: name 'cv2' is not defined

In [3]:
# Replace with your dataset name
dataset_name = ["pfo_val", "pfo_train"]
for dataset_name in dataset_name:
    # Remove from DatasetCatalog
    if dataset_name in DatasetCatalog.list():
        DatasetCatalog.remove(dataset_name)

    # Remove from MetadataCatalog
    if dataset_name in MetadataCatalog.list():
        MetadataCatalog.remove(dataset_name)


In [8]:
def get_my_dataset_dicts(img_dir, annotation_json):
    with open(annotation_json) as f:
        coco = json.load(f)

    image_id_to_file = {img["id"]: img["file_name"] for img in coco["images"]}
    image_id_to_size = {img["id"]: (img["height"], img["width"]) for img in coco["images"]}

    dataset_dicts = []
    for img_id, file_name in image_id_to_file.items():
        record = {}
        record["file_name"] = os.path.join(img_dir, file_name)
        record["image_id"] = img_id
        height, width = image_id_to_size[img_id]
        record["height"] = height
        record["width"] = width

        record["annotations"] = []
        for ann in coco["annotations"]:
            if ann["image_id"] != img_id:
                continue

            obj = {
                "bbox": ann["bbox"],
                "bbox_mode": BoxMode.XYWH_ABS,
                "category_id": ann["category_id"]
            }
            record["annotations"].append(obj)

        dataset_dicts.append(record)

    return dataset_dicts


DatasetCatalog.register("pfo_val", lambda: get_my_dataset_dicts('/home/tarobben/scratch/RCNN/PFO_png/', '/home/tarobben/scratch/RCNN/pfo_val.json' ))

MetadataCatalog.get("pfo_val").thing_classes = ["ROI"]

In [ ]:
# Get dataset
dataset_dicts = DatasetCatalog.get("pfo_train")
metadata = MetadataCatalog.get("pfo_train")

# Show 3 random images with bounding boxes
for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])

    visualizer = Visualizer(img[:, :, ::-1], metadata=metadata, scale=0.5)
    out = visualizer.draw_dataset_dict(d)

    # If you're in a notebook (e.g., Jupyter), use this:
    from IPython.display import display
    from PIL import Image
    display(Image.fromarray(out.get_image()[:, :, ::-1]))

In [6]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("pfo_train",)
cfg.DATASETS.TEST = ()  # or ("pfo_val",) if you registered a validation set
cfg.DATALOADER.NUM_WORKERS = 8

cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 6
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 30000
cfg.SOLVER.STEPS = []

cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # you only have one class ("ROI")

cfg.OUTPUT_DIR = "/home/tarobben/scratch/RCNN/output_pfo"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()


[04/10 15:37:51 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (2, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, we

[04/10 15:37:52 d2.engine.train_loop]: Starting training from iteration 0
[04/10 15:38:19 d2.utils.events]:  eta: 10:55:32  iter: 19  total_loss: 0.908  loss_cls: 0.6638  loss_box_reg: 0.000918  loss_rpn_cls: 0.2333  loss_rpn_loc: 0.0102  time: 1.3153  data_time: 0.0621  lr: 4.9953e-06  max_mem: 19166M
[04/10 15:38:46 d2.utils.events]:  eta: 11:21:13  iter: 39  total_loss: 0.7498  loss_cls: 0.5388  loss_box_reg: 0.0007815  loss_rpn_cls: 0.214  loss_rpn_loc: 0.009834  time: 1.3377  data_time: 0.0149  lr: 9.9902e-06  max_mem: 19166M
[04/10 15:39:13 d2.utils.events]:  eta: 11:28:01  iter: 59  total_loss: 0.5623  loss_cls: 0.3484  loss_box_reg: 0.0006597  loss_rpn_cls: 0.2005  loss_rpn_loc: 0.008272  time: 1.3421  data_time: 0.0133  lr: 1.4985e-05  max_mem: 19166M
[04/10 15:39:40 d2.utils.events]:  eta: 11:26:18  iter: 79  total_loss: 0.4309  loss_cls: 0.2224  loss_box_reg: 0.0004821  loss_rpn_cls: 0.1892  loss_rpn_loc: 0.008888  time: 1.3372  data_time: 0.0177  lr: 1.998e-05  max_mem: 191

In [13]:
# Look at training curves in tensorboard:
%reload_ext tensorboard
%tensorboard --logdir output_pfo

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
TensorFlow installation not found - running with reduced feature set.
Address already in use
Port 6006 is in use by another program. Either identify and stop that program, or start the server with a different port.

In [9]:
# Get dataset
dataset_dicts = DatasetCatalog.get("pfo_val")
metadata = MetadataCatalog.get("pfo_val")
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator("pfo_val", output_dir="/home/tarobben/scratch/RCNN/")
val_loader = build_detection_test_loader(cfg, "pfo_val")
print(inference_on_dataset(predictor.model, val_loader, evaluator))

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (2, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, we

[04/11 11:51:00 d2.evaluation.coco_evaluation]: Trying to convert 'pfo_val' to COCO format ...
[04/11 11:51:00 d2.data.datasets.coco]: Converting annotations of dataset 'pfo_val' to COCO format ...)
[04/11 11:51:00 d2.data.datasets.coco]: Converting dataset dicts into COCO format
[04/11 11:51:00 d2.data.datasets.coco]: Conversion finished, #images: 1328, #annotations: 190
[04/11 11:51:00 d2.data.datasets.coco]: Caching COCO format annotations at '/home/tarobben/scratch/RCNN/pfo_val_coco_format.json' ...
[04/11 11:51:01 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[04/11 11:51:01 d2.data.common]: Serializing 1328 elements to byte tensors and concatenating them all ...
[04/11 11:51:01 d2.data.common]: Serialized dataset takes 0.21 MiB
[04/11 11:51:01 d2.evaluation.evaluator]: Start inference on 1328 batches


/home/tarobben/.conda/envs/detectron2/lib/python3.8/site-packages/detectron2/structures/image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride


[04/11 11:51:03 d2.evaluation.evaluator]: Inference done 11/1328. Dataloading: 0.0018 s/iter. Inference: 0.0791 s/iter. Eval: 0.0004 s/iter. Total: 0.0813 s/iter. ETA=0:01:47
[04/11 11:51:08 d2.evaluation.evaluator]: Inference done 61/1328. Dataloading: 0.0021 s/iter. Inference: 0.0964 s/iter. Eval: 0.0005 s/iter. Total: 0.0992 s/iter. ETA=0:02:05
[04/11 11:51:13 d2.evaluation.evaluator]: Inference done 110/1328. Dataloading: 0.0021 s/iter. Inference: 0.0981 s/iter. Eval: 0.0005 s/iter. Total: 0.1009 s/iter. ETA=0:02:02
[04/11 11:51:18 d2.evaluation.evaluator]: Inference done 158/1328. Dataloading: 0.0021 s/iter. Inference: 0.0995 s/iter. Eval: 0.0005 s/iter. Total: 0.1022 s/iter. ETA=0:01:59
[04/11 11:51:23 d2.evaluation.evaluator]: Inference done 206/1328. Dataloading: 0.0022 s/iter. Inference: 0.1003 s/iter. Eval: 0.0005 s/iter. Total: 0.1031 s/iter. ETA=0:01:55
[04/11 11:51:28 d2.evaluation.evaluator]: Inference done 252/1328. Dataloading: 0.0021 s/iter. Inference: 0.1017 s/iter. E